<a href="https://colab.research.google.com/github/na2027/CE888/blob/main/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [13]:
col_names = ['JokeCount'] + [f'Joke{num + 1}' for num in range(100)]
df_jokes = pd.read_excel('/content/jester-data-1.xls',header=None, names=col_names)


In [14]:
df_jokes.head()

,JokeCount,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [15]:
print(df_jokes.loc[11740, 'Joke29'])

-2.04


In [16]:
df_jokes.shape

(24983, 101)

In [17]:
df_jokes = df_jokes.drop(columns=['JokeCount'])


In [21]:
orig_arr = df_jokes.values

Labeling 10% of the dataset cells as NaNs (represented as 99 in our case) to create a validation set.


In [28]:
new_arr, validation_indices = replace_99(orig_arr, 0.1)


In [30]:
 """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  """
def replace_99(orig, percentage=0.1):
  new_data = orig.copy()
  rated = np.where(orig_arr!=99)
  len_rated = len(rated[0])
  idx = np.random.choice(len_rated, size=int(percentage*len_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

3. Infer hidden ratings of the users and hidden characteristics of the jokes by utilizing latent factors modeling.


In [31]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [32]:
 """ Predict a rating given a user_id and an item_id.
    """
def predict_joke_rating(user_id, item_id):
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

In [37]:
def train_data(user_id, item_id, rating, alpha=0.0001):
    prediction_rating = predict_joke_rating(user_id, item_id)
    rating_err =  prediction_rating - rating
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

In [38]:
 """ Iterate over all rows for few iterations
    """
def sgd(iterations=40):
    for iteration in range(iterations):
        err_lst = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train_data(user_id, item_id, rating)
                    err_lst.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [ ]:
sgd()

26.21618978648504
23.29328825672223
21.16196526170869
18.29527925237508
17.336137372805794


4. Comparing predictions (obtained through latent factors modeling) with true values (original dataset).

In [39]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
compare_data = pd.DataFrame(values)
compare_data.columns = [f'Joke{num + 1}' for num in range(100)]

In [40]:
compare_data

,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,Joke40,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,"(-7.82, 0.19300342999660502)","(8.79, 0.03910340555348782)","(99.0, 0.07814027825838722)","(-8.16, 0.3463025925795551)","(-7.52, 0.5719143778604618)","(99.0, 0.38847553390606965)","(99.0, 0.3769008382200199)","(4.17, 0.530645990520482)","(-8.98, 0.1462158798474346)","(-4.76, 0.47641643644053266)","(-8.5, 0.5358183044918642)","(-6.75, 0.5832569571885852)","(-7.18, 0.24349566748288254)","(99.0, 0.25268820709547224)","(-7.18, 0.0721691454151952)","(-7.52, 0.20142537028622737)","(-7.43, 0.6306267972136296)","(-9.81, 0.5505848799238353)","(-9.85, 0.5653833254320023)","(-9.85, 0.16132278768950248)","(-9.37, 0.4903243584243533)","(1.5, 0.386042774021842)","(-4.37, 0.20405920150544946)","(-9.81, 0.5975667126448264)","(-8.5, 0.2689164059977431)","(1.12, 0.2318670792282351)","(7.82, 0.29832195396383265)","(2.86, 0.12428355224754477)","(9.13, 0.3551418314789165)","(-7.43, 0.4818060775169028)","(2.14, 0.28860381265831564)","(-4.08, 0.19431554551720118)","(99.0, 0.49195005833959066)","(7.82, 0.19633473409929408)","(5.05, 0.6345209256872539)","(4.95, 0.27182605116094094)","(99.0, 0.6082333016437278)","(-8.4, 0.2268411395301646)","(-8.4, 0.4449014525396983)","(-8.4, 0.3158131328818729)",...,"(8.59, 0.25949764701266437)","(3.59, 0.3433939101811697)","(-6.84, 0.30908582172598376)","(99.0, 0.6086609205249663)","(2.82, 0.49075941622609726)","(-1.36, 0.14952715338371)","(-9.08, 0.25829208896263806)","(8.3, 0.5875439547477714)","(5.68, 0.5960617813960934)","(99.0, 0.2661747498715993)","(99.0, 0.5574512165088956)","(99.0, 0.06620896052070271)","(99.0, 0.14124355547585363)","(99.0, 0.3142264536723386)","(99.0, 0.5742916487526964)","(99.0, 0.6071536100985059)","(99.0, 0.07037748577817476)","(-9.42, 0.4669105368099495)","(99.0, 0.4040740927219805)","(99.0, 0.4124571425461831)","(99.0, 0.13456726675322972)","(-7.72, 0.39143325469753415)","(99.0, 0.26230601043564483)","(99.0, 0.3936486330661762)","(99.0, 0.008089404304768464)","(99.0, 0.595314762589417)","(99.0, 0.463221918762695)","(99.0, 0.03842744848685369)","(99.0, 0.12659291034578363)","(99.0, 0.2705492184168692)","(2.82, 0.18608203588768596)","(99.0, 0.13869671659039098)","(99.0, 0.5051476014493288)","(99.0, 0.03639831976267621)","(99.0, 0.4665101693241423)","(99.0, 0.1970590312909584)","(-5.63, 0.12138871933310769)","(99.0, 0.29645840603063356)","(99.0, 0.5760896164463016)","(99.0, 0.400630005560059)"
1,"(4.08, 0.40658547978955145)","(-0.29, 0.2517883178041211)","(6.36, 0.5631999361702311)","(4.37, 0.5817690417312695)","(-2.38, 1.0642431776333918)","(-9.66, 0.9018288512542242)","(-0.73, 0.894986346074224)","(-5.34, 0.9395986274586101)","(8.88, 0.7906506769113967)","(9.22, 0.8951473621281774)","(6.75, 1.1623592553359179)","(8.64, 0.7441687868111921)","(4.42, 0.5764774462209207)","(7.43, 0.27832728710192534)","(4.56, 0.11674969350733183)","(-0.97, 0.5336463623759576)","(4.66, 1.232810556033016)","(-0.68, 0.5832299665652018)","(3.3, 0.6725992059161399)","(-1.21, 0.1903981218297937)","(0.87, 0.47574196341515623)","(8.64, 0.5042494910313393)","(8.35, 0.5968694490161168)","(9.17, 1.047189324357846)","(0.05, 0.5861905493152645)","(7.57, 0.6926774537185328)","(4.71, 0.5669955288550026)","(0.87, 0.2594748837135085)","(-0.39, 0.7629391110593762)","(6.99, 0.6343600621370782)","(6.5, 0.48539848233369826)","(-0.92, 0.7715263281704339)","(7.14, 0.964609433177741)","(9.03, 0.8395188321688882)","(-1.8, 1.227789053949661)","(99.0, 0.6356888839822455)","(7.09, 1.14

In [41]:
print(compare_data.loc[11740, 'Joke29'])



(-2.04, 0.3816745325821559)


In [45]:
orig_data = orig_arr
assert len(validation_indices[0]) == len(validation_indices[1])
validation_size = len(validation_indices[0])
errors = []
for i in range(validation_size):
  user_id = validation_indices[0][i]
  joke_id = validation_indices[1][i]
  prediction_rating = predict_joke_rating(user_id, joke_id)
  real_rating = original_data[user_id][joke_id]
  error = prediction_rating - real_rating
  errors.append(error)
mse = (np.array(errors) ** 2).mean()
print('Joke Validation MSE:', mse)

Joke Validation MSE: 27.72800907501605


Making predictions on the test set.


In [48]:
fill_data = np.zeros(orig_data.shape)
rows, columns = orig_data.shape
for user_id in range(rows):
  for joke_id in range(columns):
    if orig_data[user_id][joke_id] == 99:
      fill_data[user_id][joke_id] = predict_joke_rating(user_id, joke_id)
    else:
      fill_data[user_id][joke_id] = orig_data[user_id][joke_id]


In [49]:
fill_data

array([[-7.82      ,  8.79      , -9.66      , ...,  0.29645841,
         0.57608962,  0.40063001],
       [ 4.08      , -0.29      ,  6.36      , ...,  0.34      ,
        -4.32      ,  1.07      ],
       [ 0.32099412,  0.08437559,  0.17546333, ...,  0.45894759,
         0.90603503,  0.62016733],
       ...,
       [ 0.29673833,  0.16742589,  0.37260354, ...,  0.26656143,
         0.59672146,  0.3599589 ],
       [ 0.31835615,  0.17915852,  0.39865516, ...,  0.28680014,
         0.64144476,  0.38729081],
       [ 2.43      ,  2.67      , -3.98      , ...,  0.35864223,
         0.82772653,  0.48421795]])

Finding the best and worst rated jokes.

In [51]:
avg = np.mean(fill_data, axis=0)

In [52]:
max_joke = -9999999999
max_joke_index = -1
min_joke = 99999999999
min_joke_index = -1

for i in range(len(avg)):
  if avg[i] > max_joke:
    max_joke = avg[i]
    max_joke_index = i
  elif avg[i] < min_joke:
    min_joke = avg[i]
    min_joke_index = i

print(f'The best-rated joke is {max_joke_index + 1}, with a avg rating of: {max_joke}')
print(f'The worst-rated joke is {min_joke_index + 1}, with a avg rating of: {min_joke}')

The best-rated joke is 50, with a avg rating of: 3.6635799317045312
The worst-rated joke is 16, with a avg rating of: -3.103724712535571
